In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.base import TransformerMixin,BaseEstimator
from sklearn import cross_validation, metrics

from sklearn.manifold import TSNE, TSNE, MDS

/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn.decomposition import pca

In [4]:
from sklearn import linear_model
from sklearn.neural_network import BernoulliRBM
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
import  sklearn.neural_network as nn
import sklearn.decomposition as dec
from sklearn import svm
import sklearn.ensemble as ens

/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
import os

In [7]:
os.chdir('dineral')

In [8]:
from internaldata import Database, Classifier

/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [9]:
clf = Classifier()

In [12]:
os.chdir('..')

In [30]:
db=Database()
data = db.load_data()

In [31]:
data.head()

,Hash,Datum,Text,Lastschrift,Deleted,Kategorie,Unnamed: 6,Unnamed: 7
0,3c3550b40b42a3367a2cb4b29d28412d,2014-01-03,01-37897-1 100000013241880130751593563\nAssur...,176.75,False,Krankenkasse,NaN,NaN
1,868f4d0ba29c129f408f979f3a9f8e0e,2014-01-03,BARGELDBEZUG\nVOM 01.01.2014\nKARTEN NR. 64186...,200.00,True,NaN,NaN,NaN
2,6848fb24b03ed71e56d98fd8f3af3401,2014-01-03,Zürcher Kantonalbank CH2300700350040352767 K...,35.00,False,Anschaffungen,NaN,NaN
3,5c42bb51a03c0c035f074a9a9b8ddcbb,2014-01-03,KAUF/DIENSTLEISTUNG\nVOM 31.12.2013\nKARTEN NR...,58.00,False,Ausgang,NaN,NaN
4,cf97222b176cb612026a061b0757d230,2014-01-05,ÜBERTRAG\nAUS KONTO 92-900275-2\nSCHOCH TOBIAS...,-1000.00,False,Sparen,NaN,NaN


In [32]:
class Scaler(BaseEstimator,TransformerMixin):
    
    def fit(self, X, y=None, **fit_params):
        self._factor = np.max(np.abs(X))
        return self
    
    def transform(self, X, y=None, **fit_params):
        return np.matrix((X/self._factor)).T

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

In [33]:
class ItemSelector(BaseEstimator,TransformerMixin):
    
    def __init__(self, key=0):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return data[self.key]

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

In [34]:
class DenseTransformer(BaseEstimator,TransformerMixin):

        def transform(self, X, y=None, **fit_params):
            return X.todense()

        def fit_transform(self, X, y=None, **fit_params):
            self.fit(X, y, **fit_params)
            return self.transform(X)

        def fit(self, X, y=None, **fit_params):
            return self

In [35]:
features = FeatureUnion([('text',Pipeline([('select',ItemSelector('Text')),
                                           ('vect',CountVectorizer(analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                                           ('trans',TfidfTransformer(use_idf=True))])),
                         ('number',Pipeline([('select',ItemSelector('Lastschrift')),('scale',Scaler())]))])

In [36]:
features = FeatureUnion([('text',Pipeline([('select',ItemSelector('Text')),
                                           ('vect',CountVectorizer(analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                                           ('trans',TfidfTransformer(use_idf=True))]))])

In [37]:
features = Pipeline([('vect',CountVectorizer(ngram_range=(4,5),analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                     ('trans',TfidfTransformer(use_idf=True))])

In [38]:
data['Kategorie'] = data.Kategorie.cat.add_categories([u'Delete'])

In [39]:
data.Kategorie[data.Kategorie.isnull()]=u'Delete'

In [40]:
data.Kategorie[data.Deleted]=np.nan

In [41]:
categories = data.Kategorie.cat.categories
target = data.Kategorie.cat.codes

In [42]:
data_train, data_test, target_train, target_test = cross_validation.train_test_split(data, target, test_size=0.05)

In [43]:
text_clf = Pipeline([('features',features),('clf',SGDClassifier(loss='modified_huber',penalty='l2',random_state=12))])

In [44]:
parameters = {
              'features__vect__ngram_range':[(i,j) for j in range(3,7) for i in range(1,j)],
              'clf__alpha': np.linspace(3e-3,1e-4,20)
             }

In [45]:
gs_clf = GridSearchCV(text_clf,parameters,cv=5,verbose=True, n_jobs=8)
gs_clf.fit(data_train['Text'],target_train)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits


/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('features', Pipeline(steps=[('vect', CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 5), preprocessor=No...     penalty='l2', power_t=0.5, random_state=12, shuffle=True, verbose=0,
       warm_start=False))]),
       fit_params={}, iid=True, n_jobs=8,
       param_grid={'features__vect__ngram_range': [(1, 3), (2, 3), (1, 4), (2, 4), (3, 4), (1, 5), (2, 5), (3, 5), (4, 5), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6)], 'clf__alpha': array([0.003  , 0.00285, 0.00269, 0.00254, 0.00239, 0.00224, 0.00208,
       0.00193, 0.00178, 0.00163, 0.00147, 0.00132, 0.00117, 0.00102,
       0.00086, 0.00071, 0.00056, 0.00041, 0.00025, 0.0001 ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=True)

In [46]:
print gs_clf.best_params_

{'features__vect__ngram_range': (2, 5), 'clf__alpha': 0.0010157894736842104}


In [47]:
preds = gs_clf.predict(data_test['Text'])

In [48]:
preds_all = gs_clf.predict(data['Text'])

In [49]:
print metrics.classification_report(target_test,preds)

             precision    recall  f1-score   support

         -1       1.00      1.00      1.00         8
          0       0.25      0.33      0.29         3
          1       1.00      0.50      0.67         2
          2       0.85      0.85      0.85        13
          3       0.50      1.00      0.67         2
          5       1.00      1.00      1.00         2
          6       0.83      0.83      0.83         6
          7       0.93      0.97      0.95        29
          8       0.00      0.00      0.00         0
          9       1.00      0.75      0.86         4
         10       0.00      0.00      0.00         1
         11       1.00      1.00      1.00         4
         12       1.00      0.50      0.67         4
         13       1.00      1.00      1.00         2
         14       1.00      1.00      1.00         5
         15       1.00      1.00      1.00         2
         16       0.82      0.69      0.75        13
         17       1.00      1.00      1.00   

/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/tobi/numerics/envs/test_dineral/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [63]:
import datetime

In [64]:
text_clf = gs_clf.best_estimator_
text_clf.classes_names = categories

text_clf.TRAINING_DATE = datetime.datetime.now()
text_clf.TRAINING_SAMPLES = data_train.shape[0]
text_clf.TEST_SAMPLES = data_test.shape[0]
text_clf.SCORE = metrics.precision_recall_fscore_support(target_test,preds)

In [65]:
import pickle
with open(clf.properties,"wb+") as fp:
    pickle.dump(text_clf,fp)

In [96]:
tsne = TSNE(init='pca',verbose=True,perplexity=30,n_iter=2000,n_iter_without_progress=50,learning_rate=1500,
            early_exaggeration=5)
f_red = tsne.fit_transform(features.fit_transform(data.Text).toarray())

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 1539
[t-SNE] Computed conditional probabilities for sample 1539 / 1539
[t-SNE] Mean sigma: 0.408142
[t-SNE] Error after 100 iterations with early exaggeration: 0.681182
[t-SNE] Error after 175 iterations: 0.620439
